In [ ]:
#Set up paths
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature



from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri


# Some other useful packages 
import importlib
import copy
import time
import cftime
import yaml

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(MkP)
importlib.reload(GrU)





In [ ]:
with open('configure_ana_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

season = cfg['plot']['season']
valData  = cfg['validation']['data']
valYears = cfg['validation']['years']


######################################################

ymdPat = '2004-*' #cfg[x]['ymdPat']
print( ymdPat , season )
print( valData , valYears )


In [ ]:

x='c153_ne30_x5' #_smth'
exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
print( f"A: exp={exp} from {subd}, SrcGrid={Src}, Hkey={Hkey}, user={useri} ")
A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , Src=Src, Hkey=Hkey, 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )

x='c153_ne120_x2' #_smth'
exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
print( f"B: exp={exp} from {subd}, SrcGrid={Src}, Hkey={Hkey}, user={useri} ")
B = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd ,  Src=Src, Hkey=Hkey, 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )

x='ne240x2' #_QxQsst' #_smth'
exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
print( f"C: exp={exp} from {subd}, SrcGrid={Src}, Hkey={Hkey}, user={useri} ")
C = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd ,  Src=Src, Hkey=Hkey, 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )

## Thsi is here because it is the best source a parameterized GW info
x='c64_rdgres' #_smth'
exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
ymdPat1=cfg[x]['ymdPat']
print( f"D: exp={exp} from {subd}, SrcGrid={Src}, Hkey={Hkey}, user={useri} ")
D = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , Src=Src, Hkey=Hkey, 
                     hsPat='cam.h1i' , ymdPat=ymdPat1,verbose=True, open_dataset=True )



In [ ]:
print( C.Src )
Dst='fv1x1'
RgObs={}
RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=A.Src, dst=Dst)
RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=B.Src, dst=Dst)
RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=C.Src, dst=Dst)


In [ ]:
#season='jun'
UUc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='U',return_time=True)
UUb_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='U',return_time=True)
UUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='U',return_time=True)

OUc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='OMEGAU',return_time=True)
OUb_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='OMEGAU',return_time=True)
OUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='OMEGAU',return_time=True)

Oc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='OMEGA',return_time=True)
Ob_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='OMEGA',return_time=True)
Oa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='OMEGA',return_time=True)

OUpa_s = OUa_s - Oa_s*UUa_s
OUpb_s = OUb_s - Ob_s*UUb_s
OUpc_s = OUc_s - Oc_s*UUc_s





In [ ]:
########################
# Some massaging
########################


print( f'{A.exp} Years {len(A.X.time) /12. }')
print( f'{B.exp} Years {len(B.X.time) /12. }')
print( f'{C.exp} Years {len(C.X.time) /12. }')

RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=A.Src, dst=Dst)
UUa_s_x1=RgF.Horz(xfld_Src=UUa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
OUpa_s_x1=RgF.Horz(xfld_Src=OUpa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=B.Src, dst=Dst)
UUb_s_x1=RgF.Horz(xfld_Src=UUb_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
OUpb_s_x1=RgF.Horz(xfld_Src=OUpb_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=C.Src, dst=Dst)
UUc_s_x1=RgF.Horz(xfld_Src=UUc_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
OUpc_s_x1=RgF.Horz(xfld_Src=OUpc_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

##
#ps_c =Dc.PS.values
lev_c=C.X.lev.values
zlev_c=-7.0*np.log( lev_c/1_000. )

hyai_c=C.X.hyai.values
hybi_c=C.X.hybi.values
hyam_c=C.X.hyam.values
hybm_c=C.X.hybm.values

#ps_x =Dx.PS.values
lev_a=A.X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )

lev_b=B.X.lev.values
zlev_b=-7.0*np.log( lev_b/1_000. )


lat1,lon1=GrU.latlon( grid=Dst )

In [ ]:
UUc_sz=np.average( UUc_s_x1 , axis=2 )
UUb_sz=np.average( UUb_s_x1 , axis=2 )
UUa_sz=np.average( UUa_s_x1 , axis=2 )


OUpa_sz=np.average( OUpa_s_x1 , axis=2 )
OUpb_sz=np.average( OUpb_s_x1 , axis=2 )
OUpc_sz=np.average( OUpc_s_x1 , axis=2 )



In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=0.5*np.linspace( -0.1,0.1,num=21) 
dlev=21 #np.linspace( -20,20,num=21) 
cmap='gist_ncar'
exp_A=A.exp
fldoo="$\overline{u'\omega'}$"

n=1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat1,zlev_a,OUpa_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat1,zlev_a,OUpa_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat1,zlev_b,OUpb_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat1,zlev_b,OUpb_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{B.exp}> \n {season.upper()} {years_b[0]}-{years_b[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat1,zlev_c,OUpc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat1,zlev_c,OUpc_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{C.exp}> \n {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)



"""
Axes1 = Pu.axes_def(n=4,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""

In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=0.5*np.linspace( -0.1,0.1,num=21) 
dlev=21 #np.linspace( -20,20,num=21) 
cmap='gist_ncar'
exp_A=A.exp
fldoo="$\overline{u'\omega'}$"

n=1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lon1,lat1,OUpa_s_x1[11,:,:] ,levels=clev, cmap=cmap )
#co2=ax1.contour(lat_a,zlev_a,OUpa_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" )
#ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
n=2
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lon1,lat1,OUpb_s_x1[11,:,:] ,levels=clev, cmap=cmap )
#co2=ax1.contour(lat_a,zlev_a,OUpa_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{B.exp}> \n {season.upper()} {years_b[0]}-{years_b[-1]}" )
#ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
n=3
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lon1,lat1,OUpc_s_x1[11,:,:] ,levels=clev, cmap=cmap )
#co2=ax1.contour(lat_a,zlev_a,OUpa_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{C.exp}> \n {season.upper()} {years_c[0]}-{years_c[-1]}" )
#ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

"""
n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_b,zlev_b,OUpb_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_b,zlev_b,OUpb_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{exp_B}> \n {season.upper()} {years_b[0]}-{years_b[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,OUpc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,OUpc_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{exp_C}> \n {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""


"""
Axes1 = Pu.axes_def(n=4,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""

In [ ]:
##

In [ ]:
#from myPythonTools.Plotting import LatLonMaps as LL 
# Cartopy for pretty maps
clev=0.5*np.linspace( -0.1,0.1,num=51) 

fields=[ OUpa_s_x1[11,:,:], OUpb_s_x1[11,:,:], OUpc_s_x1[11,:,:] ]#,scale*aa_s[:,:]  ]

cmaps = ['gist_ncar','gist_ncar','gist_ncar', ] #
cmaps = ['bwr',]  #'gist_ncar','gist_ncar', ] #

clevs=[clev,clev,clev   ] # ,clev,clev,clev/2]


titles= [ f"{fldoo} <{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
            f"{fldoo} <{B.exp}> \n {season.upper()} {years_b[0]}-{years_b[-1]}" ,
            f"{fldoo} <{C.exp}> \n {season.upper()} {years_c[0]}-{years_c[-1]}" ]

MapProj = ccrs.PlateCarree(central_longitude=180.)

print( " got MapProj " )
LL.Maps( fields=fields ,lons=[lon1,lon1,lon1] ,lats=[lat1,lat1,lat1] ,clevs=clevs,cmaps=cmaps , titles=titles,  Projection=MapProj )

In [ ]:
##

In [ ]:
#####################################
# Kluge up param GW tau's 
#####################################


taurdgx = D.X.TAUARDGBETAX.values
taurdgy = D.X.TAUARDGBETAY.values
taumm=D.X.TAU_MOVMTN.values


taurdgm=np.sqrt( taurdgx**2 + taurdgy**2 )

taurdgm.shape

RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=D.Src, dst=Dst)
taurdgm_x1=RgF.Horz(xfld_Src=taurdgm, Src=Src, Dst=Dst , RegridObj_In= RegridObj )
taumm_x1=RgF.Horz(xfld_Src=taumm , Src=Src, Dst=Dst , RegridObj_In= RegridObj )



ataurdgm_x1=10*np.average( taurdgm_x1, axis=0 )
ataumm_x1=10*np.average( taumm_x1, axis=0 )


In [ ]:
#from myPythonTools.Plotting import LatLonMaps as LL 
# Cartopy for pretty maps
clev=0.5*np.linspace( -0.1,0.1,num=51) 

fields=[ OUpa_s_x1[11,:,:] , OUpa_s_x1[11,:,:]  + ataumm_x1[11,:,:] + ataurdgm_x1[11,:,:] , OUpb_s_x1[11,:,:], OUpc_s_x1[11,:,:] ]#,scale*aa_s[:,:]  ]

cmaps = ['gist_ncar','gist_ncar','gist_ncar', ] #
cmaps = ['bwr',]  #'gist_ncar','gist_ncar', ] #

clevs=[clev,clev,clev , clev  ] # ,clev,clev,clev/2]

fldoo=r"$\overline{u'\omega'}$"
fldpp=r"${\tau}_{param,climo}$"
titles= [ f"{fldoo} <{A.Src}>  {season.upper()} {years_a[0]}-{years_a[-1]}" ,
         f"{fldoo}(JJA 2004)+{fldpp}" ,
            f"{fldoo} <{B.Src}>  {season.upper()} {years_b[0]}-{years_b[-1]}" ,
            f"{fldoo} <{C.Src}>  {season.upper()} {years_c[0]}-{years_c[-1]}" ]

MapProj = ccrs.PlateCarree(central_longitude=180.)

print( " got MapProj " )
LL.Maps( fields=fields ,lons=[lon1,lon1,lon1,lon1] ,lats=[lat1,lat1,lat1,lat1] ,clevs=clevs,cmaps=cmaps , titles=titles,  CoastWidth=1, Projection=MapProj )